References:
- https://platform.openai.com/docs/guides/speech-to-text/quickstart
- https://medium.com/nlplanet/transcribing-youtube-videos-with-whisper-in-a-few-lines-of-code-f57f27596a55

In [2]:
#pip install openai-whisper
#pip install pytube
import pytube as pt
from pytube.exceptions import RegexMatchError
import whisper

In [3]:
from tqdm.auto import tqdm

In [4]:
import torch  # install steps: pytorch.org
device = "cuda" if torch.cuda.is_available() else "cpu"


In [5]:
yt = pt.YouTube("https://www.youtube.com/watch?v=aJjCjWdvmfo") #video length is 3:04 
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

'd:\\thesis\\TruthfulSearch\\notebooks\\audio.mp3'

In [13]:
model = whisper.load_model("medium").to(device)



In [14]:
result = model.transcribe("audio.mp3")
print(result) 

d:\Program Files\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'text': ' What is ironic? What is incongruous? Nito nakakatawa, nakakagalit, nadaanan ko yung Freedom Park. Yung sana ang gusto namin kung konta kami rito, nagingi kami ng permit, binigyan kami. Nung araw na dati kami, gunawin yung permit at sinirahan yung Freedom Park. Walang di-administration, galing tutundang malakanyan." Binatikos ni dating Pangulong Rodrigo Duterte ang malakanyang matapos cancelahin ang kanilang permit para makapagsagawa ng rally sa Freedom Park sa Dumaguete City kagabi. Sa isang sulat na galing sa opisina ng Dumaguete City Administrator, kinansila nito ang permit ng mga supporters ni Duterte. Sumulat umano ang Dumaguete Police sa City Government na hindi nila kayang magbigay ng seguridad sa dating Pangulo at iba pang mga personalidad na dadalo sa rally. Hindi rin makapagbigay ng metal barriers ang lungsod. Sa inilabas na pahayag ng Provincial Government ng Negros Oriental, isasara muna ang Freedom Park para sa maintenance at rehabilitation. May mga heavy equipme

In [15]:
result['text']

' What is ironic? What is incongruous? Nito nakakatawa, nakakagalit, nadaanan ko yung Freedom Park. Yung sana ang gusto namin kung konta kami rito, nagingi kami ng permit, binigyan kami. Nung araw na dati kami, gunawin yung permit at sinirahan yung Freedom Park. Walang di-administration, galing tutundang malakanyan." Binatikos ni dating Pangulong Rodrigo Duterte ang malakanyang matapos cancelahin ang kanilang permit para makapagsagawa ng rally sa Freedom Park sa Dumaguete City kagabi. Sa isang sulat na galing sa opisina ng Dumaguete City Administrator, kinansila nito ang permit ng mga supporters ni Duterte. Sumulat umano ang Dumaguete Police sa City Government na hindi nila kayang magbigay ng seguridad sa dating Pangulo at iba pang mga personalidad na dadalo sa rally. Hindi rin makapagbigay ng metal barriers ang lungsod. Sa inilabas na pahayag ng Provincial Government ng Negros Oriental, isasara muna ang Freedom Park para sa maintenance at rehabilitation. May mga heavy equipment di na 

In [17]:
import translators as ts


In [18]:
ts.translate_text(result['text'], 'google', to_language = 'en')

'What is ironic? What is incongruous? Its funny, annoying, I passed the Freedom Park. That would have been what we wanted if we were here, we had a permit, we were given. The day we used to be, the permit and the Freedom Park lived. There is no non-administration, it is a great deal of Malakanyan. "Former President Rodrigo Duterte criticized Malacañang after canceling their permit to conduct a rally at Freedom Park in Dumaguete City last night. In a letter from the Dumaguete City Administrator\'s office, it recognized the Duterte\'s supporters permitted the Dumaguete police to the City Government that they could not provide security to the former president. NEGROS ORIENTAL GOVERNMENT, Freedom Park will be closed for maintenance and rehabilitation. Behind it. They did not endorse them, but the people were charged with the constitution. You can\'t stop me because that could be censorship. You can\'t stop it now whatever I say here that someone is hurt or I can speak unlawful, sedition or

In [19]:
result['segments']

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 22.0,
  'text': ' What is ironic? What is incongruous? Nito nakakatawa, nakakagalit, nadaanan ko yung Freedom Park.',
  'tokens': [50364,
   708,
   307,
   33719,
   30,
   708,
   307,
   834,
   556,
   894,
   563,
   30,
   426,
   3528,
   20332,
   39051,
   10449,
   11,
   20332,
   514,
   559,
   304,
   270,
   11,
   297,
   1538,
   29915,
   8384,
   288,
   1063,
   22208,
   4964,
   13,
   51464],
  'temperature': 0.0,
  'avg_logprob': -0.4396775828467475,
  'compression_ratio': 1.1149425287356323,
  'no_speech_prob': 0.8600757122039795},
 {'id': 1,
  'seek': 2200,
  'start': 23.0,
  'end': 49.0,
  'text': ' Yung sana ang gusto namin kung konta kami rito, nagingi kami ng permit, binigyan kami. Nung araw na dati kami, gunawin yung permit at sinirahan yung Freedom Park. Walang di-administration, galing tutundang malakanyan."',
  'tokens': [50414,
   398,
   1063,
   15490,
   2562,
   38723,
   297,
   7428,
   49304,
   

In [20]:
result['language']

'tl'

In [21]:
import os
os.remove("audio.mp3")

-------------------------------------------------------

In [33]:
id = 'aLZ85hb4wjE'
video = 'https://www.youtube.com/watch?v='+id
video

'https://www.youtube.com/watch?v=aLZ85hb4wjE'

In [34]:
yt = pt.YouTube(video) 
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

'd:\\thesis\\TruthfulSearch\\notebooks\\audio.mp3'

In [35]:
model = whisper.load_model("medium").to(device)

In [36]:
try:
    result = model.transcribe("audio.mp3")
    print(result["text"]) 
except:
    print ("oops")

 You


In [39]:
len(result['text'].split())

1

In [40]:
result

{'text': ' You',
 'segments': [{'id': 0,
   'seek': 6000,
   'start': 60.0,
   'end': 62.0,
   'text': ' You',
   'tokens': [50364, 509, 50464],
   'temperature': 0.0,
   'avg_logprob': -0.941067099571228,
   'compression_ratio': 0.2727272727272727,
   'no_speech_prob': 0.7944472432136536}],
 'language': 'en'}

----------------------------

In [3]:
import pandas as pd
import numpy as np
import csv
import os


In [11]:
import pytube as pt
import whisper
import translators as ts



In [5]:
covid_ph_df = pd.read_csv("../datasets/covid_philippines/videos.csv").drop("Unnamed: 0", axis=1)
covid_ph_df = covid_ph_df[['video_id','video_title','video_transcript']]
covid_ph_df.head()

,video_id,video_title,video_transcript
0,aLZ85hb4wjE,Normally crowded streets of Philippine capital...,NaN
1,sdsz-t540WI,Millions more unemployed in the Philippines am...,Julio's Evangelista used to earn around $300 ...
2,qERZgLv3MKg,Philippines president threatens to &#39;shoot ...,NaN
3,sYI97jv-pZg,"PH records 2,725 new COVID cases from Dec. 12 ...",NaN
4,pMUumjHY3tw,DOH reports slight rise in COVID-19 cases | ANC,NaN


i left  the originally generated video trancript to compare to whisper

In [6]:
covid_ph_df['final_transcript'] = None

In [7]:
covid_ph_df

,video_id,video_title,video_transcript,final_transcript
0,aLZ85hb4wjE,Normally crowded streets of Philippine capital...,NaN,None
1,sdsz-t540WI,Millions more unemployed in the Philippines am...,Julio's Evangelista used to earn around $300 ...,None
2,qERZgLv3MKg,Philippines president threatens to &#39;shoot ...,NaN,None
3,sYI97jv-pZg,"PH records 2,725 new COVID cases from Dec. 12 ...",NaN,None
4,pMUumjHY3tw,DOH reports slight rise in COVID-19 cases | ANC,NaN,None
...,...,...,...,...
195,MWmIqdwy4Ww,"COVID-19: China bans arrivals from UK, Belgium...",china has imposed fresh travel bans on kobe 1...,None
196,lqbQhDHBuNQ,"Philippines tallies 10,775 new COVID-19 cases",the philippines reports 10 775 new covet 19 c...,None
197,g_6Rj6cPfOs,"Philippines’ COVID 19 cases climb to 22,474; d...",Parliament was a think is bullet in neon arro...,None
198,eUxXjKan0Yg,Philippines&#39; daily COVID-19 deaths hit all...,NaN,None


In [8]:
import torch  # install steps: pytorch.org
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("medium").to(device)


In [98]:
def whisper_transcribe(input_df):
    
    df = input_df[['video_id','video_transcript']].copy()
    whis = np.array([])
    for row in range(len(input_df)):
        try:
            if pd.isna(covid_ph_df.at[row, 'video_transcript']):
                video = 'https://www.youtube.com/watch?v=' + input_df['video_id'][row]
                print("transcribing "+video)
                yt = pt.YouTube(video) 
                stream = yt.streams.filter(only_audio=True)[0]
                stream.download(filename="audio_" + input_df['video_id'][row]+".mp3")
                audio_file = ("audio_"+ input_df['video_id'][row] +".mp3")
                result = model.transcribe(audio_file)
                os.remove("audio_" + input_df['video_id'][row]+".mp3")
                if len(result['text'].split())>=10:
                    en_res = ts.translate_text(result['text'], 'google', to_language = 'en')
                    whis =np.append(whis,en_res)
                else:
                    whis =np.append(whis,float(np.nan))
            else:
                whis = np.append(whis,input_df['video_transcript'][row])

        except Exception as e:
            print("Something happened:", e)
            whis = np.append(whis,input_df['video_transcript'][row])
        

    df['final_transcript'] = whis
    return df

In [94]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[:5])

transcribing https://www.youtube.com/watch?v=aLZ85hb4wjE


d:\Program Files\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


transcribing https://www.youtube.com/watch?v=qERZgLv3MKg
transcribing https://www.youtube.com/watch?v=sYI97jv-pZg
transcribing https://www.youtube.com/watch?v=pMUumjHY3tw


In [95]:
whisper_trans_covidPH

,video_id,video_transcript,final_transcript
0,aLZ85hb4wjE,NaN,nan
1,sdsz-t540WI,Julio's Evangelista used to earn around $300 ...,Julio's Evangelista used to earn around $300 ...
2,qERZgLv3MKg,NaN,"About the problem of covid. If you like, baril..."
3,sYI97jv-pZg,NaN,The daily average of the Philippines new COVID...
4,pMUumjHY3tw,NaN,The COVID wards of several Philippine hospital...


In [99]:
final_transcript = whisper_trans_covidPH.copy()

In [100]:
covid_ph_df[5:50]

,video_id,video_title,video_transcript,final_transcript
5,DWxIvQlpJK8,Metro Manila to be placed on lockdown due to c...,NaN,None
6,E5F3xA_zkFc,Manila Lockdown: One of the longest COVID lock...,[Music] chinese from entering the answer of c...,None
7,dIsaz_XlmTw,DOH logs 18 cases of JN.1 COVID-19 variant in PH,the Department of Health on Sunday said it ha...,None
8,8dTelszbObM,Coronavirus: First death outside China reporte...,well the corona virus continues to spread at ...,None
9,MQ5aYS4YFlQ,COVID-19 In Philippines: The Starving Urban Po...,it has one of the toughest and longest lockdo...,None
10,CEIrzjA8euQ,DOH urges public to mask up amid rise in COVID...,NaN,None
11,3YFpjgIQqEo,WATCH: DOH Usec. Tayag on rise of COVID-19 cas...,NaN,None
12,587N9bJ5J5k,Philippines hospitals reeling from COVID crisis,tents have sprung up in hospital grounds all ...,None
13,nTUWK8vufOk,Asia&#39;s Lost Generation: The Hidden Cost Of...,COVID-19 has left\r\neconomies around the wor...,None
14,lw16DeB6zns,COVID-19 leads to significant job losses in th...,rambo panganiban says his family feels lucky ...,None


In [101]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[5:50].reset_index())

transcribing https://www.youtube.com/watch?v=DWxIvQlpJK8


d:\Program Files\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


transcribing https://www.youtube.com/watch?v=dIsaz_XlmTw
transcribing https://www.youtube.com/watch?v=8dTelszbObM
Something happened: 
transcribing https://www.youtube.com/watch?v=MQ5aYS4YFlQ
Something happened: The length of `query_text` exceeds the limit.
transcribing https://www.youtube.com/watch?v=CEIrzjA8euQ
Something happened: 
transcribing https://www.youtube.com/watch?v=Mxf8uGFcqSE
Something happened: 
transcribing https://www.youtube.com/watch?v=cPVE7QGS7As
transcribing https://www.youtube.com/watch?v=E56W-5xVOss
Something happened: 
transcribing https://www.youtube.com/watch?v=05JLyd58R-w
Something happened: 
transcribing https://www.youtube.com/watch?v=RhDHGgo4yZg
Something happened: 
transcribing https://www.youtube.com/watch?v=oChS6LbuluU
transcribing https://www.youtube.com/watch?v=liEUC1_l8e8
transcribing https://www.youtube.com/watch?v=o1kPskxFkQ8
transcribing https://www.youtube.com/watch?v=wJKPv-XmArQ
transcribing https://www.youtube.com/watch?v=NQeI1CRCqeo
transcribi

In [102]:
whisper_trans_covidPH

,video_id,video_transcript,final_transcript
0,DWxIvQlpJK8,NaN,Community quality is needed throughout Manila ...
1,E5F3xA_zkFc,[Music] chinese from entering the answer of c...,[Music] chinese from entering the answer of c...
2,dIsaz_XlmTw,the Department of Health on Sunday said it ha...,The Department of Health on Sunday said it has...
3,8dTelszbObM,well the corona virus continues to spread at ...,well the corona virus continues to spread at ...
4,MQ5aYS4YFlQ,it has one of the toughest and longest lockdo...,it has one of the toughest and longest lockdo...
5,CEIrzjA8euQ,NaN,nan
6,3YFpjgIQqEo,NaN,nan
7,587N9bJ5J5k,tents have sprung up in hospital grounds all ...,tents have sprung up in hospital grounds all ...
8,nTUWK8vufOk,COVID-19 has left\r\neconomies around the wor...,COVID-19 has left\r\neconomies around the wor...
9,lw16DeB6zns,rambo panganiban says his family feels lucky ...,rambo panganiban says his family feels lucky ...


In [112]:
whisper_trans_covidPH['video_transcript'][5]

nan

In [113]:
whisper_trans_covidPH['video_transcript'].isna().sum()

16

In [114]:
whisper_trans_covidPH['final_transcript'].isna().sum()

0

In [106]:
final_transcript = pd.concat([final_transcript,whisper_trans_covidPH],ignore_index=True)


In [107]:
final_transcript

,video_id,video_transcript,final_transcript
0,aLZ85hb4wjE,NaN,nan
1,sdsz-t540WI,Julio's Evangelista used to earn around $300 ...,Julio's Evangelista used to earn around $300 ...
2,qERZgLv3MKg,NaN,"About the problem of covid. If you like, baril..."
3,sYI97jv-pZg,NaN,The daily average of the Philippines new COVID...
4,pMUumjHY3tw,NaN,The COVID wards of several Philippine hospital...
5,DWxIvQlpJK8,NaN,Community quality is needed throughout Manila ...
6,E5F3xA_zkFc,[Music] chinese from entering the answer of c...,[Music] chinese from entering the answer of c...
7,dIsaz_XlmTw,the Department of Health on Sunday said it ha...,The Department of Health on Sunday said it has...
8,8dTelszbObM,well the corona virus continues to spread at ...,well the corona virus continues to spread at ...
9,MQ5aYS4YFlQ,it has one of the toughest and longest lockdo...,it has one of the toughest and longest lockdo...


In [115]:
whisper_trans_covidPH =whisper_transcribe(covid_ph_df[50:100].reset_index())

transcribing https://www.youtube.com/watch?v=3ZXR2eARmuQ


d:\Program Files\miniconda3\lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


transcribing https://www.youtube.com/watch?v=57wz3HuIVLA
Something happened: 
transcribing https://www.youtube.com/watch?v=kn6DKdInYXk
transcribing https://www.youtube.com/watch?v=lbK7UjoLr8o
transcribing https://www.youtube.com/watch?v=lczwrm68u6I
Something happened: The length of `query_text` exceeds the limit.
transcribing https://www.youtube.com/watch?v=ibWFsmcnefk
transcribing https://www.youtube.com/watch?v=e0fN7HxkIUc
transcribing https://www.youtube.com/watch?v=SLrKdYsNbSo
transcribing https://www.youtube.com/watch?v=2FgFNBIJTIA
Something happened: 2FgFNBIJTIA is age restricted, and can't be accessed without logging in.
transcribing https://www.youtube.com/watch?v=LtW0cTMWdbY
transcribing https://www.youtube.com/watch?v=hlh47Rfs6ZY
transcribing https://www.youtube.com/watch?v=y4qaoHc2ezk
transcribing https://www.youtube.com/watch?v=HK5rfm4X2G0
transcribing https://www.youtube.com/watch?v=2VRGUm_wQ0w
transcribing https://www.youtube.com/watch?v=8w61tigk0ag
transcribing https://w

In [116]:
whisper_trans_covidPH

,video_id,video_transcript,final_transcript
0,3ZXR2eARmuQ,and the Philippines has become the first coun...,And the Philippines has become the first count...
1,HLYSlKY6Ww4,[Music] the biggest challenge Hammond is the ...,[Music] the biggest challenge Hammond is the ...
2,57wz3HuIVLA,the more contagious delta variant of the coro...,the more contagious delta variant of the coro...
3,kn6DKdInYXk,there's been no let-up in the Philippines eit...,There's been no let up in the Philippines eith...
4,lbK7UjoLr8o,health workers are protesting in manila calli...,"Health workers are protesting in Manila, calli..."
5,lczwrm68u6I,[Music] hi i'm femi okay i don't need to tell...,[Music] hi i'm femi okay i don't need to tell...
6,FJEMcVK0MWI,Chinese authorities say they're investigating...,Chinese authorities say they're investigating...
7,j3E3NF9nk44,NaN,nan
8,dj5ov38ihZI,as countries evacuate their citizens from Wuh...,as countries evacuate their citizens from Wuh...
9,Ji47WRv2tQE,NaN,nan


In [ ]:
whisper_trans_covidPH.to_csv('../datasets/covid_philippines/whisper_transcript.csv')
